### Create S3 Bucket

In [8]:
import boto3
import os

In [ ]:
s3 = boto3.client("s3")

bucket_name = ""

In [ ]:
res = s3.list_buckets()
res 

In [4]:
def create_bucket(bucket_name):
    s3.create_bucket(Bucket = bucket_name)
    print("Bucket is created")

In [7]:
create_bucket(bucket_name)

Bucket is created


### Upload a single file to s3

In [11]:
def upload_file(file_path, object_name = None):
    if object_name is None:
        object_name = os.path.basename(file_path)

    s3.upload_file(file_path, bucket_name, object_name)
    print("File Uploaded")

In [12]:
upload_file("data/index.html")

File Uploaded


## List all object in a bucket

In [13]:
def list_object():
    res = s3.list_objects_v2(Bucket = bucket_name)

    for obj in res["Contents"]:
        print(obj)

In [ ]:
list_object()

### Download s3 file to local system

In [15]:
def download_file(object_name, file_path):
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))

    s3.download_file(bucket_name, object_name, file_path)
    print("Donwload Completed")

In [16]:
download_file("index.html", "data_download/index.html")

Donwload Completed


### Upload a folder to s3

In [ ]:
for root, dirs, file_names in os.walk('data'):
    print(root, dirs, file_names)

In [42]:
def upload_folder(dirs_path, s3_prefix):
    for root, dirs, file_names in os.walk('data'):
        for file in file_names:
            file_path = os.path.join(root, file).replace("\\","/")
            relpath = os.path.relpath(file_path, dirs_path)
            s3_key = os.path.join(s3_prefix, relpath).replace("\\", "/")

            s3.upload_file(file_path, bucket_name, s3_key)

In [43]:
upload_folder("data", "s3_data")

### Download S3 folder to local system

In [ ]:
list_object()

In [56]:
local_path = "s3_download" 
s3_prefix = "s3_data"

def download_dir(local_path, s3_prefix):
    os.makedirs(local_path, exist_ok = True)

    paginator = s3.get_paginator("list_objects_v2")

    for result in paginator.paginate(Bucket=bucket_name, Prefix = s3_prefix):
        if "Contents" in result:
            for key in result["Contents"]:
                s3_key = key['Key']

                local_file = os.path.join(local_path, os.path.relpath(s3_key,s3_prefix))

                s3.download_file(bucket_name, s3_key, local_file)
                print("Downlaod Completed")

In [57]:
download_dir(local_path, s3_prefix)

Downlaod Completed
Downlaod Completed
Downlaod Completed


### Delete All files in an s3 bucket

In [58]:
def delete_obj():
    res = s3.list_objects_v2(Bucket=bucket_name)

    if "Contents" in res:
        for obj in res["Contents"]:
            s3.delete_object(Bucket=bucket_name, Key=obj["Key"])
            print("Deleting")



In [59]:
delete_obj()

Deleting
Deleting
Deleting
Deleting


In [ ]:
# delete bucket
s3.delete_bucket(Bucket=bucket_name)